In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('数据包/03_个人客户流水.csv')
first = pd.read_excel('优级名单（分先后享受型）.xlsx', sheet_name='先享受型'    )
later = pd.read_excel('优级名单（分先后享受型）.xlsx', sheet_name='后享受型'    )

In [3]:
all_out = []
all_in = []
first_expense = [0 for _ in range(12)]
later_ck = [0 for _ in range(12)]
later_lc = [0 for _ in range(12)]
later_ck_money = 0
later_lc_money = 0

In [4]:
first = first.values
later = later.values
data = data.values

account_name = data[:, 0]

In [5]:
# first_ind = np.zeros(shape=(1230923, ))
# for fi in first:
#     first_ind += (account_name == fi).astype(np.int)

later_ind = np.zeros(shape=(1230923, ))
for la in later:
    later_ind += (account_name == la).astype(np.int)

In [6]:
# first_data = data[first_ind >= 1]
later_data = data[later_ind >= 1]


In [9]:
later_ck_money = np.sum(later_data[later_data[:, 3] == 'CK', 4])
later_lc_money = np.sum(later_data[later_data[:, 3] == 'GMLC', 4])

a1 = later_ck_money / (later_ck_money + later_lc_money)
a2 = later_lc_money / (later_ck_money + later_lc_money)

for ac_ in later:
    ind_ = later_data[:, 0] == ac_
    data_ = later_data[ind_]
    allin_ = np.sum(data_[data_[:, 3] == 'ZR', 4]) + np.sum(data_[data_[:, 3] == 'CK', 4])
    allout_ = np.sum(data_[:, 4]) - allin_
    all_in.append(allin_)
    all_out.append(allout_)


In [14]:
all_out = np.sort(all_out)
all_out = np.abs(all_out)
all_in = np.abs(all_in)
in_out_rate = np.median(all_out) / np.median(all_in)

In [14]:
for fi_ac in first:
    data_ = first_data[first_data[:, 0] == fi_ac]
    month = [0 for _ in range(12)]
    for da in data_:
        if da[3] == 'QTXF' or da[3] == 'GMLC':
            ind_ = (da[1] - 20180000) // 100 - 1
            if month[ind_]:
                continue
            else:
                month[ind_] = 1
                first_expense[ind_] += 1


for la_ac in later:
    data_ = later_data[later_data[:, 0] == la_ac]
    month_ck = [0 for _ in range(12)]
    month_lc = [0 for _ in range(12)]
    for da in data_:
        if da[3] == 'CK':
            ind_ = (da[1] - 20180000) // 100 - 1
            if month_ck[ind_]:
                continue
            else:
                month_ck[ind_] = 1
                later_ck[ind_] += 1
        elif da[3] == 'GMLC':
            ind_ = (da[1] - 20180000) // 100 - 1
            if month_lc[ind_]:
                continue
            else:
                month_lc[ind_] = 1
                later_lc[ind_] += 1

In [16]:
b1_list = []
c1_list = []

target_list = []

r1 = 0.003
r2 = 0.021
p1 = 0.028
p2 = 0.04

print('搜索参数范围...')
for b1 in np.arange(0.1, 1.05, 0.05):
    for c1 in np.arange(0.1, 1.05, 0.05):
        if a1 * b1 + a2 * c1 < in_out_rate:
            continue
        else:
            b1_list.append(b1)
            c1_list.append(c1)
            target = a1 * b1 * r1 + a1 * (1 - b1) * r2 + a2 * c1 * p1 + a2 * (1 - c1) * p2
            target_list.append(target)

ind = np.argmax(target_list)
b1 = b1_list[ind]
b2 = 1 - b1
c1 = c1_list[ind]
c2 = 1 - c1
print('最佳占比为：')
print('活期存款：{}'.format(a1 * b1))
print('定期存款：{}'.format(a1 * b2))
print('活期理财：{}'.format(a2 * c1))
print('定期理财：{}'.format(a2 * c2))



搜索参数范围...
最佳占比为：
活期存款：0.049213143245874946
定期存款：0.9350497216716239
活期理财：0.0007868567541250486
定期理财：0.014950278328375921
